In [ ]:
!zip -r audio_speech.zip audio_speech/
!zip -r audio_speech_csvs.zip audio_speech_csvs/

In [ ]:
!unzip audio_speech_csvs.zip

In [59]:
!rm -rf audio_speech
!mkdir -p audio_speech

!rm -rf audio_speech_csvs
!mkdir -p audio_speech_csvs

In [60]:
!mkdir audio_speech_raw

In [ ]:
!unzip /content/Audio_Speech_Actors_01-24.zip -d audio_speech_raw

In [ ]:
!pip install ffmpeg-python        # &> shell command to hide output
!pip install tensorflow-io 
!pip install python_speech_features 
!pip install librosa==0.8 
print("Packages Installed")

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, MultiHeadAttention, Reshape, Dropout, GlobalAveragePooling1D, Conv1D, LayerNormalization, Normalization
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks.Callback.
from tensorflow.keras import callbacks

from IPython.display import HTML, Audio
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import tensorflow as tf
import tensorflow_io as tfio
import matplotlib.pyplot as plt
import numpy as np
from python_speech_features import mfcc
from matplotlib import cm
import pickle
import librosa
print("Packages Imported")

In [55]:
import librosa
import tensorflow as tf
from pathlib import Path


max_shape = 150000

Emotion = {'01': 'neutral', '02' : 'calm', '03' : 'happy', 
         '04': 'sad', '05': 'angry', '06':'fearful', 
          '07': 'disgust', '08': 'surprised'}

def change_file(file_name, csv_dir):
  only_file_name = file_name.split('/')[-1]
  actor_id = file_name.split('/')[-2]
  file_ids = only_file_name.split('-')

  Path(f"{csv_dir}/{actor_id}").mkdir(parents=True, exist_ok=True)

  # print(file_ids)
  emotion = Emotion[file_ids[2]]
  new_file_name = f'{emotion}_{only_file_name}'

  data , sampling_rate = librosa.load(file_name)
  elems_to_pad = max_shape - data.shape[0]
  if elems_to_pad > 0:
    data = np.pad(data, (0, elems_to_pad), 'constant', constant_values=(0,-1)).shape
  else:
    print('NO PADDING')

  np.savetxt(f"{csv_dir}/{actor_id}/{new_file_name}.csv", data, delimiter=",")
  return new_file_name, actor_id


In [ ]:
import glob
import subprocess

audio_files = glob.glob("audio_speech_raw/*/*.wav")

for file_path in audio_files:
  new_file_name, actor_id = change_file(file_path, 'audio_speech_csvs')
  Path(f"audio_speech/{actor_id}").mkdir(parents=True, exist_ok=True)
  # print(new_file_name)
  print(f"{file_path}", f"audio_speech/{actor_id}/{new_file_name}")
  subprocess.run(["cp", f"{file_path}", f"audio_speech/{actor_id}/{new_file_name}"])
  # break